In [1]:
%matplotlib inline
import time
import imp
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
import unidecode



In [2]:
from problem import get_train_data

X_df, y_array = get_train_data()

# keeping first 2 numbers of APE
t = time.time()
X_df['Activity_code (APE)'] = X_df['Activity_code (APE)'].fillna(value = '00').astype('str').apply(lambda x: x[:2])

# infer zipcode from city

if(True):
    X_df['Zipcode'] = X_df['Zipcode'].astype('str').apply(lambda x: x[:2])
    city_names = X_df[['Zipcode', 'City']].groupby('Zipcode').City.apply(list).values
    zipcodes = X_df[['Zipcode', 'City']].groupby('Zipcode').City.apply(list).index
    zip_dict = {zipcode:list(set(cities)) for zipcode,cities in zip(zipcodes, city_names)}
    
    #remove nans from city lists
    
    for key in zip_dict.keys():
        for i, city in enumerate(zip_dict[key]):
            if type(city) != type('a string'):
                zip_dict[key].pop(i)
           
    # infer zip by checking lists
    
    def infer_zip(city):
        for key in zip_dict:
            if city in zip_dict[key]:
                return key
    
    X_df.loc[X_df['Zipcode'] == 'na', 'Zipcode'] = X_df.loc[X_df['Zipcode'] == 'na', 'City'].apply(infer_zip)
    X_df.loc[X_df['Zipcode'] == 'na', 'Zipcode'] = '00'
    X_df['Zipcode'] = X_df['Zipcode'].fillna('00')
    
# fill name and lower

X_df['Name'] = X_df['Name'].fillna('nan').str.lower().astype('str').apply(unidecode.unidecode)

# Fill headcounts

if(True):
    headcounts = X_df[['Name', 'Headcount']].groupby('Name').mean().fillna(0).to_dict()['Headcount']
    names = X_df['Name'].values
    heads = np.zeros(len(names))

    for i,name in enumerate(names):
        if type(name) == type('a string'):
            heads[i] = headcounts[name]

    X_df['Headcount'] = heads

# unicode and lower

X_df['City'] = X_df['City'].fillna('nan').str.lower().apply(unidecode.unidecode)

# Drop Address

award = pd.read_csv('data/award_notices_RAMP.csv.zip', compression='zip')

city_list = [unidecode.unidecode(i.lower()) for i in list(set(X_df['City'].values))]

def cedex_remover(s):
    s = s.replace('-', ' ').replace(',', ' ')
    tokens = s.split(' ')
    for k in range(len(tokens)):
        for j in range(1, len(tokens)-k + 1):
            if ' '.join(tokens[k:k+j]) in city_list:
                return ' '.join(tokens[k:k+j])
    return None

award.head(3)

print(award.shape)

# unidecode incumbent_name and lower

award['incumbent_name'] = award['incumbent_name'].str.lower().astype('str').apply(unidecode.unidecode)

# preprocess incumbent_city and fill nans
award['incumbent_city'] = award['incumbent_city'].str.lower().astype('str').apply(unidecode.unidecode).apply(cedex_remover).fillna('nan')


# Remove NA in department of provision and keep first 2
award = award[award['Departments_of_publication'].notna()]
award['Departments_of_publication'] = award['Departments_of_publication'].astype('str').apply(lambda x: x.split(',')[0][:2])


# city mean FAN

city_mean_fan = award[['incumbent_city','amount']].groupby('incumbent_city').quantile(0.5).fillna(0).to_dict()['amount']

city_lower_mean = award[['incumbent_city','amount']].groupby('incumbent_city').quantile(0.25).fillna(0).to_dict()['amount']
city_higher_mean = award[['incumbent_city','amount']].groupby('incumbent_city').quantile(0.75).fillna(0).to_dict()['amount']

# zip mean FAN

zip_mean_fan = award[['Departments_of_publication','amount']].groupby('Departments_of_publication').quantile(0.5).fillna(0).to_dict()['amount']

zip_lower_mean = award[['Departments_of_publication','amount']].groupby('Departments_of_publication').quantile(0.25).fillna(0).to_dict()['amount']
zip_higher_mean = award[['Departments_of_publication','amount']].groupby('Departments_of_publication').quantile(0.75).fillna(0).to_dict()['amount']


# APE mean FAN
company_list = [unidecode.unidecode(str(i).lower()) for i in list(set(X_df['Name'].values))]

company_awards = award[award['incumbent_name'].isin(company_list)]

ape_companies = X_df[['Name', 'Activity_code (APE)']].groupby('Name')['Activity_code (APE)'].apply(lambda x: list(x)[0])
ape_companies.index = ape_companies.index.str.lower()
ape_companies = ape_companies.to_dict()

def infer_ape(name):
    return ape_companies[name]
company_awards['APE'] = company_awards['incumbent_name'].apply(infer_ape)

ape_mean_fan = company_awards[['APE','amount']].groupby('APE').quantile(0.5).fillna(0).to_dict()['amount']
ape_lower_mean = company_awards[['APE','amount']].groupby('APE').quantile(0.25).fillna(0).to_dict()['amount']
ape_higher_mean = company_awards[['APE','amount']].groupby('APE').quantile(0.75).fillna(0).to_dict()['amount']

# Actual FAN revenue

fan_revenue = company_awards[['incumbent_name','amount']].groupby('incumbent_name').sum().fillna(0).to_dict()['amount']

# Insert features into X_df

# Ape FAN
if(True):
    def infer_ape_fan(APE):
        try:
            return ape_mean_fan[APE]
        except:
            return 0

    def infer_low_ape_fan(APE):
        try:
            return ape_lower_mean[APE]
        except:
            return 0

    def infer_high_ape_fan(APE):
        try:
            return ape_higher_mean[APE]
        except:
            return 0



    X_df['APE_fan_0.5'] = X_df['Activity_code (APE)'].apply(infer_ape_fan)
    X_df['APE_fan_0.25'] = X_df['Activity_code (APE)'].apply(infer_low_ape_fan)
    X_df['APE_fan_0.75'] = X_df['Activity_code (APE)'].apply(infer_high_ape_fan)

# City FAN
if(True):

    def infer_city_fan(city):
        try:
            return city_mean_fan[city]
        except:
            return 0

    def infer_low_city_fan(city):
        try:
            return city_lower_mean[city]
        except:
            return 0

    def infer_high_city_fan(city):
        try:
            return city_higher_mean[city]
        except:
            return 0


    X_df['city_fan_0.5'] = X_df['City'].apply(infer_city_fan)
    X_df['city_fan_0.25'] = X_df['City'].apply(infer_low_city_fan)
    X_df['city_fan_0.75'] = X_df['City'].apply(infer_high_city_fan)

# Zip_fan
if(True):
    def infer_zip_fan(zipcode):
        try:
            return zip_mean_fan[zipcode]
        except:
            return 0

    def infer_low_zip_fan(zipcode):
        try:
            return zip_lower_mean[zipcode]
        except:
            return 0

    def infer_high_zip_fan(zipcode):
        try:
            return zip_higher_mean[zipcode]
        except:
            return 0

    X_df['zip_fan_0.5'] = X_df['Zipcode'].apply(infer_zip_fan)
    X_df['zip_fan_0.25'] = X_df['Zipcode'].apply(infer_low_zip_fan)
    X_df['zip_fan_0.75'] = X_df['Zipcode'].apply(infer_high_zip_fan)

# Actual FAN
if(True):
    def infer_fan(name):
        try:
            return fan_revenue[name]
        except:
            return 0

    X_df['FAN'] = X_df['Name'].apply(infer_fan)


to_dummy = ['Activity_code (APE)', 'Zipcode', 'Year']
to_drop = ['Name', 'Address', 'City', 'Fiscal_year_end_date']
X_df = X_df.drop(to_drop, axis = 1)
X_df['y'] = y_array











C:\Users\adeik\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(304098, 28)


C:\Users\adeik\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
from binascii import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32


def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]



X_train, X_test = split_train_test_by_id(X_df, 0.2, 'Legal_ID')
#y_train = X_train['y']
X_train = X_train.drop(['Legal_ID'], axis = 1)

#y_test = X_test['y']
X_test = X_test.drop(['Legal_ID'], axis = 1)

In [4]:
y_train = X_train['y'].to_numpy()

X_train['y'] = np.max(np.array([np.ones(y_train.shape[0]), y_train]), axis = 0)

In [5]:
from auto_ml import Predictor

In [ ]:
column_descriptions = {
    'y': 'output',
    'Activity_code (APE)': 'categorical',
    'Zipcode': 'categorical',
    'Year': 'categorical'
}

ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)

ml_predictor.train(X_train, take_log_of_y = True, cv = 2,
                   model_names = ['Lasso', 'GradientBoostingRegressor', 'Ridge', 'XGBRegressor'])

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.9
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{}
Running basic data cleaning
Performing feature scaling
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{}


********************************************************************************************
About to run GridSearchCV on the pipeline for several models to predict y
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed:  7.0min remaining: 11.6min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed:  8.3min remaining:  5.0min


In [8]:
ml_predictor2.score(X_test, X_test.y)

None


***********************************************
Advanced scoring metrics for the trained regression model on this particular dataset:

Here is the overall RMSE for these predictions:
185577258.04566807

Here is the average of the predictions:
1134879.8815084372

Here is the average actual value on this validation set:
4414034.472833534

Here is the median prediction:
220688.61823658837

Here is the median actual value:
289764.0

Here is the mean absolute error:
3668780.571699861

Here is the median absolute error (robust to outliers):
174794.7417764254

Here is the explained variance:
0.00989501559166206

Here is the R-squared value:
0.009585778811106005
Count of positive differences (prediction > actual):
131267
Count of negative differences:
168680
Average positive difference:
445150.50978338235
Average negative difference:
-6177413.766729418


***********************************************




-185577258.04566807

In [6]:
column_descriptions = {
    'y': 'output',
    'Activity_code (APE)': 'categorical',
    'Zipcode': 'categorical',
    'Year': 'categorical'
}

ml_predictor2 = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)

ml_predictor2.train(X_train, take_log_of_y = True, cv = 2,
                   model_names = ['GradientBoostingRegressor'])

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.9
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingRegressor to predict y
Started at:
2019-11-

[226] random_holdout_set_from_training_data's score is: -2.376
[229] random_holdout_set_from_training_data's score is: -2.375
[232] random_holdout_set_from_training_data's score is: -2.375
[235] random_holdout_set_from_training_data's score is: -2.375
[238] random_holdout_set_from_training_data's score is: -2.374
[241] random_holdout_set_from_training_data's score is: -2.374
[244] random_holdout_set_from_training_data's score is: -2.374
[247] random_holdout_set_from_training_data's score is: -2.373
[250] random_holdout_set_from_training_data's score is: -2.373
[255] random_holdout_set_from_training_data's score is: -2.372
[260] random_holdout_set_from_training_data's score is: -2.372
[265] random_holdout_set_from_training_data's score is: -2.371
[270] random_holdout_set_from_training_data's score is: -2.371
[275] random_holdout_set_from_training_data's score is: -2.37
[280] random_holdout_set_from_training_data's score is: -2.37
[285] random_holdout_set_from_training_data's score is: -